In [23]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd
import collections
import re
import pprint as pp
import numpy as np
import matplotlib.pyplot as plt

import math
import gzip
import pickle as pkl
import tensorflow as tf
import matplotlib.pyplot as plt
from datetime import datetime
import random
random.seed(123)
import os
import time
import tflearn

import multiprocessing as mp
from multiprocessing.pool import ThreadPool
from multiprocessing import Process

from keras import Sequential
from keras.layers import Dense,Dropout,BatchNormalization, LeakyReLU
from keras import optimizers
from keras import losses

from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.feature_extraction.text import CountVectorizer 

import tensorflow as tf

import itertools
import gzip
import pickle as pkl

import fonctions
import batch_processing
import metric_loading

from os import listdir
from tqdm.notebook import tqdm
from os.path import isfile, join

from scipy.sparse import hstack
from keras import backend as K
from keras import callbacks
from sklearn.metrics import log_loss
from tensorflow import ones_like, equal, log
from keras import backend as K
from tensorflow.python import mul
import tensorflow as tf

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
def read_batch(directory, doc_name):
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep=',', header=0, index_col=False,low_memory=False)
    df = df.drop(df.columns[ [0] ], axis=1)
    df.index = df.tweet_id+'-'+df.engaging_user_id
    
    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']    
     
    X = df.drop( ['mean_daily_author_presence','std_daily_author_presence',
                  'language','tweet_id','tweet_timestamp','tweet_utctimestamp',
                  'engaged_with_user_account_creation', 'text_tokens',
                  'present_links','present_domains','hashtags',
                  'engaging_user_account_creation', 'present_media',
                  'engaged_with_user_id', 'engaging_user_id' ],axis=1)
    
#     X = X[ ['engaged_with_user_follower_count','engaged_with_user_following_count',
#     'engaged_with_user_is_verified','engaging_user_follower_count','engaging_user_following_count',
#     'engaging_user_is_verified','engagee_follows_engager',
#     'hour_ 00','hour_ 01','hour_ 02','hour_ 03','hour_ 04',
#     'hour_ 05','hour_ 06','hour_ 07','hour_ 08','hour_ 09',
#     'hour_ 10','hour_ 11','hour_ 12','hour_ 13','hour_ 14',
#     'hour_ 15','hour_ 16','hour_ 17','hour_ 18','hour_ 19','hour_ 20',
#     'hour_ 21','hour_ 22','hour_ 23','day_0','day_1','day_2','day_3', 'day_4','day_5', 'day_6',
#     'user_account_age','user_presence','author_account_age',
#     'author_presence','tweet_type_Quote','tweet_type_Retweet','tweet_type_TopLevel',
#     'photo','video','gif','nb_hashtags','nb_links','nb_domains','len_tweet',
#     'taste_rt_author','taste_rtc_author',
#     'taste_rpl_author','taste_lk_author','taste_rt_hashtag',
#     'taste_rtc_hashtag','taste_rpl_hashtag','taste_lk_hashtag',
#     'content_link','content_text','content_photo_video','content_gif',
#     'user_0','user_1','user_2','user_3',
#     'user_4','user_5','user_6','user_7',
#     'user_8','user_9','user_10','user_11' ] ]
    
    X = shuffle(X, random_state=0)
    
    nan_cols = [i for i in X.columns if X[i].isnull().any() ]

    #########################################################
    ###### no a priori model
    
    X1 = X[  ~(X[ nan_cols ].notnull().all(1) ) ]
#     y1= X1.filter( labels, axis=1 )
#     X1 = X1.drop(labels,axis=1)
    
    ##########################################################
    ###### a priori model
    
    X2 = X[  X[ nan_cols ].notnull().all(1) ]
#     y2 = X2.filter(labels, axis=1)
#     X2 = X2.drop(labels,axis=1)

    return X1, X2

def save_model(model_id, classifier):
    
    with gzip.open('/home/maxime/Desktop/RecSys2020/modeles/model_multilabel_NN_{}.pkl.gz'.format(model_id),'wb') as f:
        pkl.dump(classifier,f)
        
    return True

def build_multilabel_NN(nb):
    
    classifier = Sequential()
    classifier.add(Dense( nb+20, input_dim=nb ,activation='relu' ))
    classifier.add(Dropout(0.3))
    classifier.add(Dense( nb,activation='relu' ))
    classifier.add(Dense( nb-15,activation='relu' ))
    classifier.add(Dense( nb-30,activation='relu' ))
    classifier.add(Dense( 15,activation='relu' ))
    classifier.add(Dense(4, activation='sigmoid'))
    opt = optimizers.Adam()
    classifier.compile(optimizer = opt, loss=[custom_loss] ) 
    
    return classifier

def custom_loss(y_true,y_pred):
    
    substr = y_pred[:,1] - y_pred[:,2]
    min_value = K.constant(-1, substr.dtype.base_dtype)
    zeros = K.constant(0, substr.dtype.base_dtype)
    x = K.clip(substr, min_value, zeros)
   
    bce = tf.keras.losses.BinaryCrossentropy()
    
    total_loss =  bce(y_true, y_pred) +  100000 * K.sum(  K.square(x) ) 
    
    return total_loss

def train_on_chunk(chunk, model_id, classifier1,classifier2, nb_epochs):

    directory = '/home/maxime/Desktop/RecSys2020/data/processed_batches1/'
    sc = StandardScaler()

    for _ in range(nb_epochs):

        iteration=1
        for batch_file in chunk:

            print(iteration)
            X1, y1, X2, y2 = read_batch(directory, batch_file)
            
            #########################################################
            ###### train no a priori model
            
            X1 = X1.drop(['user_0', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6',
                          'user_7', 'user_8', 'user_9', 'user_10', 'user_11',
                          'content_link', 'content_text', 'content_photo_video', 'content_gif',
                          'taste_rt_author','taste_rtc_author','taste_rpl_author','taste_lk_author',
                          'taste_rt_hashtag','taste_rtc_hashtag','taste_rpl_hashtag','taste_lk_hashtag'],axis=1)
            
            X1 = sc.fit_transform(X1) 
            classifier1.fit(X1, y1, verbose=1 )
    
            ##########################################################
            ###### train a priori model
        
            X2 = sc.fit_transform(X2) 
            classifier2.fit(X2, y2, verbose=1 )
            
            iteration=iteration+1
            
    save_model(model_id, classifier1)
    save_model(str(model_id)+'_wpriori', classifier2)

    return classifier1,classifier2

In [28]:
%%time

batch_path='/home/maxime/Desktop/RecSys2020/data/processed_batches1/'
chunk = [f for f in listdir(batch_path) if isfile(join(batch_path, f))]

model_id=5005

classifier1 = build_multilabel_NN(86)
classifier2 = build_multilabel_NN(110)

classifier1, classifier2 = train_on_chunk(chunk[:500], model_id, classifier1,classifier2, 1)

1
Epoch 1/1
21273/21273 [==============================] - 2s 77us/step - loss: 284.6654
Epoch 1/1
8727/8727 [==============================] - 1s 110us/step - loss: 0.2760
2
Epoch 1/1
21172/21172 [==============================] - 1s 55us/step - loss: 0.5157
Epoch 1/1
8828/8828 [==============================] - 1s 65us/step - loss: 0.2337
3
Epoch 1/1
21418/21418 [==============================] - 1s 56us/step - loss: 0.3171
Epoch 1/1
8582/8582 [==============================] - 1s 84us/step - loss: 0.2162
4
Epoch 1/1
20908/20908 [==============================] - 1s 58us/step - loss: 0.2839
Epoch 1/1
9092/9092 [==============================] - 1s 79us/step - loss: 0.2183
5
Epoch 1/1
21389/21389 [==============================] - 1s 64us/step - loss: 0.2793
Epoch 1/1
8611/8611 [==============================] - 1s 69us/step - loss: 0.2082
6
Epoch 1/1
21298/21298 [==============================] - 1s 59us/step - loss: 0.2746
Epoch 1/1
8702/8702 [==============================] - 1s 74

8492/8492 [==============================] - 1s 78us/step - loss: 0.1825
49
Epoch 1/1
21387/21387 [==============================] - 2s 72us/step - loss: 0.2440
Epoch 1/1
8613/8613 [==============================] - 0s 54us/step - loss: 0.1862
50
Epoch 1/1
21321/21321 [==============================] - 1s 60us/step - loss: 0.2414
Epoch 1/1
8679/8679 [==============================] - 1s 71us/step - loss: 0.2019
51
Epoch 1/1
20931/20931 [==============================] - 1s 48us/step - loss: 0.2323
Epoch 1/1
9069/9069 [==============================] - 0s 55us/step - loss: 0.1926
52
Epoch 1/1
21689/21689 [==============================] - 1s 54us/step - loss: 0.2405
Epoch 1/1
8311/8311 [==============================] - 0s 55us/step - loss: 0.1881
53
Epoch 1/1
21333/21333 [==============================] - 1s 45us/step - loss: 0.2404
Epoch 1/1
8667/8667 [==============================] - 0s 50us/step - loss: 0.1886
54
Epoch 1/1
21152/21152 [==============================] - 1s 44us/step

Epoch 1/1
8568/8568 [==============================] - 1s 70us/step - loss: 0.1926
97
Epoch 1/1
21069/21069 [==============================] - 1s 54us/step - loss: 0.2395
Epoch 1/1
8931/8931 [==============================] - 1s 63us/step - loss: 0.1814
98
Epoch 1/1
21235/21235 [==============================] - 1s 55us/step - loss: 0.2374
Epoch 1/1
8765/8765 [==============================] - 1s 63us/step - loss: 0.1818
99
Epoch 1/1
21231/21231 [==============================] - 1s 54us/step - loss: 0.2404
Epoch 1/1
8769/8769 [==============================] - 1s 67us/step - loss: 0.1845
100
Epoch 1/1
20942/20942 [==============================] - 1s 53us/step - loss: 0.2335
Epoch 1/1
9058/9058 [==============================] - 1s 63us/step - loss: 0.1879
101
Epoch 1/1
21213/21213 [==============================] - 1s 53us/step - loss: 0.2381
Epoch 1/1
8787/8787 [==============================] - 1s 62us/step - loss: 0.1840
102
Epoch 1/1
21330/21330 [==============================] -

21250/21250 [==============================] - 1s 55us/step - loss: 0.2374
Epoch 1/1
8750/8750 [==============================] - 1s 70us/step - loss: 0.1810
145
Epoch 1/1
21381/21381 [==============================] - 1s 54us/step - loss: 0.2388
Epoch 1/1
8619/8619 [==============================] - 1s 64us/step - loss: 0.1837
146
Epoch 1/1
21200/21200 [==============================] - 1s 53us/step - loss: 0.2366
Epoch 1/1
8800/8800 [==============================] - 1s 61us/step - loss: 0.1857
147
Epoch 1/1
21387/21387 [==============================] - 1s 54us/step - loss: 0.2371
Epoch 1/1
8613/8613 [==============================] - 1s 69us/step - loss: 0.1827
148
Epoch 1/1
21507/21507 [==============================] - 1s 54us/step - loss: 0.2383
Epoch 1/1
8493/8493 [==============================] - 1s 67us/step - loss: 0.1868
149
Epoch 1/1
20940/20940 [==============================] - 1s 54us/step - loss: 0.2350
Epoch 1/1
9060/9060 [==============================] - 1s 66us/st

8982/8982 [==============================] - 1s 60us/step - loss: 0.1889
192
Epoch 1/1
21137/21137 [==============================] - 2s 71us/step - loss: 0.2357
Epoch 1/1
8863/8863 [==============================] - 1s 88us/step - loss: 0.1822
193
Epoch 1/1
21029/21029 [==============================] - 1s 58us/step - loss: 0.2379
Epoch 1/1
8971/8971 [==============================] - 1s 107us/step - loss: 0.1793
194
Epoch 1/1
21287/21287 [==============================] - 2s 79us/step - loss: 0.2383
Epoch 1/1
8713/8713 [==============================] - 1s 80us/step - loss: 0.1913
195
Epoch 1/1
21077/21077 [==============================] - 1s 46us/step - loss: 0.2347
Epoch 1/1
8923/8923 [==============================] - 0s 55us/step - loss: 0.1785
196
Epoch 1/1
21042/21042 [==============================] - 1s 43us/step - loss: 0.2351
Epoch 1/1
8958/8958 [==============================] - 0s 51us/step - loss: 0.1948
197
Epoch 1/1
21452/21452 [==============================] - 1s 44

...

9056/9056 [==============================] - 1s 70us/step - loss: 0.1809
287
Epoch 1/1
21152/21152 [==============================] - 1s 54us/step - loss: 0.2342
Epoch 1/1
8848/8848 [==============================] - 1s 62us/step - loss: 0.1824
288
Epoch 1/1
21010/21010 [==============================] - 1s 54us/step - loss: 0.2288
Epoch 1/1
8990/8990 [==============================] - 1s 61us/step - loss: 0.1823
289
Epoch 1/1
21286/21286 [==============================] - 1s 59us/step - loss: 0.2357
Epoch 1/1
8714/8714 [==============================] - 1s 72us/step - loss: 0.1847
290
Epoch 1/1
21140/21140 [==============================] - 1s 57us/step - loss: 0.2371
Epoch 1/1
8860/8860 [==============================] - 1s 73us/step - loss: 0.1804
291
Epoch 1/1
21274/21274 [==============================] - 1s 57us/step - loss: 0.2355
Epoch 1/1
8726/8726 [==============================] - 1s 73us/step - loss: 0.1836
292
Epoch 1/1
21288/21288 [==============================] - 1s 62u

21112/21112 [==============================] - 1s 61us/step - loss: 0.2306
Epoch 1/1
8888/8888 [==============================] - 1s 63us/step - loss: 0.1792
335
Epoch 1/1
21189/21189 [==============================] - 1s 56us/step - loss: 0.2364
Epoch 1/1
8811/8811 [==============================] - 1s 67us/step - loss: 0.1839
336
Epoch 1/1
21136/21136 [==============================] - 1s 55us/step - loss: 0.2325
Epoch 1/1
8864/8864 [==============================] - 1s 61us/step - loss: 0.1785
337
Epoch 1/1
21200/21200 [==============================] - 1s 49us/step - loss: 0.2347
Epoch 1/1
8800/8800 [==============================] - 0s 57us/step - loss: 0.1867
338
Epoch 1/1
21296/21296 [==============================] - 1s 64us/step - loss: 0.2331
Epoch 1/1
8704/8704 [==============================] - 1s 59us/step - loss: 0.1832
339
Epoch 1/1
20842/20842 [==============================] - 1s 62us/step - loss: 0.2334
Epoch 1/1
9158/9158 [==============================] - 1s 58us/st

8795/8795 [==============================] - 1s 59us/step - loss: 0.1776
382
Epoch 1/1
21142/21142 [==============================] - 1s 50us/step - loss: 0.2322
Epoch 1/1
8858/8858 [==============================] - 0s 52us/step - loss: 0.1768
383
Epoch 1/1
21027/21027 [==============================] - 1s 53us/step - loss: 0.2302
Epoch 1/1
8973/8973 [==============================] - 1s 79us/step - loss: 0.1810
384
Epoch 1/1
20931/20931 [==============================] - 1s 59us/step - loss: 0.2302
Epoch 1/1
9069/9069 [==============================] - 1s 61us/step - loss: 0.1783
385
Epoch 1/1
21064/21064 [==============================] - 1s 55us/step - loss: 0.2334
Epoch 1/1
8936/8936 [==============================] - 1s 85us/step - loss: 0.1851
386
Epoch 1/1
21187/21187 [==============================] - 1s 51us/step - loss: 0.2301
Epoch 1/1
8813/8813 [==============================] - 1s 57us/step - loss: 0.1837
387
Epoch 1/1
21184/21184 [==============================] - 1s 47u

20927/20927 [==============================] - 1s 49us/step - loss: 0.2275
Epoch 1/1
9073/9073 [==============================] - 1s 57us/step - loss: 0.1796
430
Epoch 1/1
21346/21346 [==============================] - 1s 48us/step - loss: 0.2298
Epoch 1/1
8654/8654 [==============================] - 0s 55us/step - loss: 0.1745
431
Epoch 1/1
20967/20967 [==============================] - 1s 49us/step - loss: 0.2343
Epoch 1/1
9033/9033 [==============================] - 1s 58us/step - loss: 0.1746
432
Epoch 1/1
21223/21223 [==============================] - 1s 50us/step - loss: 0.2349
Epoch 1/1
8777/8777 [==============================] - 1s 60us/step - loss: 0.1831
433
Epoch 1/1
21318/21318 [==============================] - 1s 52us/step - loss: 0.2331
Epoch 1/1
8682/8682 [==============================] - 0s 53us/step - loss: 0.1864
434
Epoch 1/1
21271/21271 [==============================] - 1s 48us/step - loss: 0.2326
Epoch 1/1
8729/8729 [==============================] - 1s 58us/st

8930/8930 [==============================] - 0s 56us/step - loss: 0.1766
477
Epoch 1/1
21494/21494 [==============================] - 1s 65us/step - loss: 0.2349
Epoch 1/1
8506/8506 [==============================] - 1s 63us/step - loss: 0.1906
478
Epoch 1/1
21431/21431 [==============================] - 1s 57us/step - loss: 0.2341
Epoch 1/1
8569/8569 [==============================] - 0s 57us/step - loss: 0.1779
479
Epoch 1/1
21081/21081 [==============================] - 1s 56us/step - loss: 0.2320
Epoch 1/1
8919/8919 [==============================] - 1s 57us/step - loss: 0.1801
480
Epoch 1/1
21217/21217 [==============================] - 1s 58us/step - loss: 0.2350
Epoch 1/1
8783/8783 [==============================] - 0s 55us/step - loss: 0.1819
481
Epoch 1/1
21110/21110 [==============================] - 1s 68us/step - loss: 0.2354
Epoch 1/1
8890/8890 [==============================] - 1s 99us/step - loss: 0.1823
482
Epoch 1/1
21213/21213 [==============================] - 1s 56u

In [43]:
def read_val_batch(directory, doc_name):
    
    df = pd.read_csv(directory+doc_name, encoding="utf-8", sep=',', header=0, index_col=False,low_memory=False)
    df = df.drop(df.columns[ [0] ], axis=1)
    df.index = df.tweet_id+'-'+df.engaging_user_id
    
    labels= ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']
    
    X = df.drop( ['mean_daily_author_presence','std_daily_author_presence',
                  'language','tweet_id','tweet_timestamp','tweet_utctimestamp',
                  'engaged_with_user_account_creation', 'text_tokens',
                  'present_links','present_domains','hashtags',
                  'engaging_user_account_creation', 'present_media',
                  'engaged_with_user_id', 'engaging_user_id' ],axis=1)
    
#     X = X[ labels+['engaged_with_user_follower_count','engaged_with_user_following_count',
#     'engaged_with_user_is_verified','engaging_user_follower_count','engaging_user_following_count',
#     'engaging_user_is_verified','engagee_follows_engager',
#     'hour_ 00','hour_ 01','hour_ 02','hour_ 03','hour_ 04',
#     'hour_ 05','hour_ 06','hour_ 07','hour_ 08','hour_ 09',
#     'hour_ 10','hour_ 11','hour_ 12','hour_ 13','hour_ 14',
#     'hour_ 15','hour_ 16','hour_ 17','hour_ 18','hour_ 19','hour_ 20',
#     'hour_ 21','hour_ 22','hour_ 23','day_0','day_1','day_2','day_3', 'day_4','day_5', 'day_6',
#     'user_account_age','user_presence','author_account_age',
#     'author_presence','tweet_type_Quote','tweet_type_Retweet','tweet_type_TopLevel',
#     'photo','video','gif','nb_hashtags','nb_links','nb_domains','len_tweet',
#     'taste_rt_author','taste_rtc_author',
#     'taste_rpl_author','taste_lk_author','taste_rt_hashtag',
#     'taste_rtc_hashtag','taste_rpl_hashtag','taste_lk_hashtag',
#     'content_link','content_text','content_photo_video','content_gif',
#     'user_0','user_1','user_2','user_3',
#     'user_4','user_5','user_6','user_7',
#     'user_8','user_9','user_10','user_11' ] ]
    
    X = shuffle(X, random_state=0)
    
    nan_cols = [i for i in X.columns if X[i].isnull().any() ]

    
    #########################################################
    ###### no a priori model
    
    X1 = X[  ~(X[ nan_cols ].notnull().all(1) ) ]
    y1 = X1.filter(labels, axis=1)
    X1.drop(labels,axis=1)
    
    ##########################################################
    ###### a priori model
    
    X2 = X[  X[ nan_cols ].notnull().all(1) ]
    y2 = X2.filter(labels, axis=1)
    X2.drop(labels,axis=1)

    return X1,y1,X2,y2

def validation_on_chunk(model_id, chunk, chunk_id,classifier1,classifier2):
    
    engagements = ['like_timestamp','retweet_timestamp','retweet_with_comment_timestamp','reply_timestamp']

    sc = StandardScaler()
    directory = '/home/maxime/Desktop/RecSys2020/data/pr_test/'
    iteration=1
    
    for test_batch in chunk:
        print(str(iteration)+' '+test_batch)
        
        X1, X2 = read_batch(directory, test_batch)

        ##########################################################
        ###### train no a priori model
        
        X1 = X1.drop(['user_0', 'user_1', 'user_2', 'user_3', 'user_4', 'user_5', 'user_6',
                      'user_7', 'user_8', 'user_9', 'user_10', 'user_11',
                      'content_link', 'content_text', 'content_photo_video', 'content_gif',
                      'taste_rt_author','taste_rtc_author','taste_rpl_author','taste_rt_hashtag',
                      'taste_rtc_hashtag', 'taste_rpl_hashtag', 'taste_lk_author','taste_lk_hashtag'  ],axis=1)
   
        print(len(X1.index))
        index1 = X1.index
        X1 = sc.fit_transform(X1) 
        y_pred1 = classifier1.predict(X1, verbose=1) 
        y_pred1 = y_pred1.astype('float64')
        
        d1 = {'Tweet_Id': [ x.split('-')[0] for x in index1 ], 'User_Id': [ x.split('-')[1] for x in index1 ],
              'like_timestamp':y_pred1[:,0],'retweet_timestamp':y_pred1[:,1],
              'retweet_with_comment_timestamp':y_pred1[:,2],'reply_timestamp':y_pred1[:,3] }
#      
# 'truth_like_timestamp':y1.like_timestamp,'truth_retweet_timestamp':y1.retweet_timestamp,
#  'truth_retweet_with_comment_timestamp':y1.retweet_with_comment_timestamp,'truth_reply_timestamp':y1.reply_timestamp

        result1 = pd.DataFrame(data=d1)
        
        #########################################################
        ###### train a priori model
        
        print(len(X2.index))
        index2 = X2.index
        X2 = sc.fit_transform(X2) 
        y_pred2 = classifier2.predict(X2) 
        y_pred2 =  y_pred2.astype('float64')
        
        d2 = {'Tweet_Id': [ x.split('-')[0] for x in index2 ], 'User_Id': [ x.split('-')[1] for x in index2 ],
              'like_timestamp':y_pred2[:,0],'retweet_timestamp':y_pred2[:,1],
              'retweet_with_comment_timestamp':y_pred2[:,2],'reply_timestamp':y_pred2[:,3]}

#               'truth_like_timestamp':y2.like_timestamp,'truth_retweet_timestamp':y2.retweet_timestamp,
#               'truth_retweet_with_comment_timestamp':y2.retweet_with_comment_timestamp,'truth_reply_timestamp':y2.reply_timestamp 

        result2 = pd.DataFrame(data=d2)

        resultat = pd.concat([result1,result2], axis=0)

        for idx, engagement in enumerate(engagements):
            result = resultat.filter(['Tweet_Id','User_Id', engagement,'truth_'+str(engagement) ],axis=1) 
            result.columns = ['Tweet_Id','User_Id', 'Prediction'] 
            
            result.to_csv('/home/maxime/Desktop/RecSys2020/data/validation_results/article_{}_{}.csv'.format(engagement, model_id),
            header=None, index=None, sep=',', mode='a')
        
        iteration=iteration+1
  
    return True

In [34]:
eval_batch_path='/home/maxime/Desktop/RecSys2020/data/val_pr_batches2'
eval_batch_list = [f for f in listdir(eval_batch_path) if isfile(join(eval_batch_path, f))]
model_id = 5005

validation_on_chunk(model_id, eval_batch_list, 0,classifier1,classifier2)

1 pr_job_992109.tsv
22295
22295/22295 [==============================] - 0s 19us/step
7705
2 pr_job_992066.tsv
22315
22315/22315 [==============================] - 0s 22us/step
7685
3 pr_job_991893.tsv
22011
22011/22011 [==============================] - 0s 21us/step
7989
4 pr_job_991760.tsv
22172
22172/22172 [==============================] - 0s 21us/step
7828
5 pr_job_992103.tsv
22169
22169/22169 [==============================] - 0s 22us/step
7831
6 pr_job_991897.tsv
22110
22110/22110 [==============================] - 0s 21us/step
7890
7 pr_job_991706.tsv
21908
21908/21908 [==============================] - 0s 20us/step
8092
8 pr_job_992044.tsv
22044
22044/22044 [==============================] - 0s 20us/step
7956
9 pr_job_992082.tsv
22076
22076/22076 [==============================] - 0s 21us/step
7924
10 pr_job_992032.tsv
22159
22159/22159 [==============================] - 1s 23us/step
7841
11 pr_job_991927.tsv
22073
22073/22073 [==============================] - 0s 20us/step
79

22050
22050/22050 [==============================] - 1s 25us/step
7950
91 pr_job_992034.tsv
22021
22021/22021 [==============================] - 0s 21us/step
7979
92 pr_job_991778.tsv
21964
21964/21964 [==============================] - 0s 21us/step
8036
93 pr_job_992054.tsv
22116
22116/22116 [==============================] - 0s 20us/step
7884
94 pr_job_991736.tsv
22199
22199/22199 [==============================] - 0s 21us/step
7801
95 pr_job_991848.tsv
21780
21780/21780 [==============================] - 0s 19us/step
8220
96 pr_job_992073.tsv
22147
22147/22147 [==============================] - 0s 22us/step
7853
97 pr_job_991781.tsv
21928
21928/21928 [==============================] - 0s 18us/step
8072
98 pr_job_992006.tsv
21775
21775/21775 [==============================] - 0s 17us/step
8225
99 pr_job_992049.tsv
21943
21943/21943 [==============================] - 0s 20us/step
8057
100 pr_job_991856.tsv
22169
22169/22169 [==============================] - 0s 18us/step
7831
101 pr_j

22110/22110 [==============================] - 0s 19us/step
7890
179 pr_job_991768.tsv
21967
21967/21967 [==============================] - 1s 25us/step
8033
180 pr_job_992088.tsv
22143
22143/22143 [==============================] - 0s 20us/step
7857
181 pr_job_991695.tsv
22073
22073/22073 [==============================] - 0s 19us/step
7927
182 pr_job_991771.tsv
21958
21958/21958 [==============================] - 0s 17us/step
8042
183 pr_job_991994.tsv
21946
21946/21946 [==============================] - 0s 18us/step
8054
184 pr_job_992084.tsv
22183
22183/22183 [==============================] - 0s 16us/step
7817
185 pr_job_991680.tsv
22040
22040/22040 [==============================] - 0s 16us/step
7960
186 pr_job_991908.tsv
22164
22164/22164 [==============================] - 0s 17us/step
7836
187 pr_job_991807.tsv
21807
21807/21807 [==============================] - 1s 29us/step
8193
188 pr_job_991986.tsv
21765
21765/21765 [==============================] - 0s 22us/step
8235
189 p

22115/22115 [==============================] - 0s 18us/step
7885
267 pr_job_991737.tsv
22009
22009/22009 [==============================] - 0s 17us/step
7991
268 pr_job_991882.tsv
22232
22232/22232 [==============================] - 0s 20us/step
7768
269 pr_job_991720.tsv
22031
22031/22031 [==============================] - 1s 30us/step
7969
270 pr_job_991903.tsv
22054
22054/22054 [==============================] - 0s 16us/step
7946
271 pr_job_991871.tsv
22099
22099/22099 [==============================] - 0s 18us/step
7901
272 pr_job_992076.tsv
21805
21805/21805 [==============================] - 0s 18us/step
8195
273 pr_job_991742.tsv
22181
22181/22181 [==============================] - 0s 17us/step
7819
274 pr_job_992081.tsv
22003
22003/22003 [==============================] - 0s 17us/step
7997
275 pr_job_991759.tsv
22099
22099/22099 [==============================] - 0s 18us/step
7901
276 pr_job_991735.tsv
21923
21923/21923 [==============================] - 0s 17us/step
8077
277 p

22010/22010 [==============================] - 0s 21us/step
7990
355 pr_job_991687.tsv
21943
21943/21943 [==============================] - 0s 21us/step
8057
356 pr_job_991732.tsv
22060
22060/22060 [==============================] - 0s 21us/step
7940
357 pr_job_991753.tsv
22096
22096/22096 [==============================] - 0s 19us/step
7904
358 pr_job_992040.tsv
22025
22025/22025 [==============================] - 0s 19us/step
7975
359 pr_job_992050.tsv
22044
22044/22044 [==============================] - 0s 19us/step
7956
360 pr_job_991691.tsv
22062
22062/22062 [==============================] - 0s 22us/step
7938
361 pr_job_991750.tsv
22032
22032/22032 [==============================] - 0s 18us/step
7968
362 pr_job_991717.tsv
22173
22173/22173 [==============================] - 0s 19us/step
7827
363 pr_job_991931.tsv
21973
21973/21973 [==============================] - 0s 20us/step
8027
364 pr_job_991711.tsv
22082
22082/22082 [==============================] - 0s 20us/step
7918
365 p

22117/22117 [==============================] - 0s 20us/step
7883
443 pr_job_991875.tsv
22108
22108/22108 [==============================] - 0s 21us/step
7892
444 pr_job_991995.tsv
21928
21928/21928 [==============================] - 0s 20us/step
8072
445 pr_job_991970.tsv
21847
21847/21847 [==============================] - 0s 19us/step
8153
446 pr_job_991914.tsv
22147
22147/22147 [==============================] - 0s 20us/step
7853
447 pr_job_992017.tsv
21705
21705/21705 [==============================] - 0s 20us/step
8295
448 pr_job_992023.tsv
21750
21750/21750 [==============================] - 0s 21us/step
8250
449 pr_job_992069.tsv
22157
22157/22157 [==============================] - 0s 20us/step
7843
450 pr_job_992122.tsv
21926
21926/21926 [==============================] - 0s 20us/step
8074


True

In [ ]:
eval_batch_path='/home/maxime/Desktop/RecSys2020/data/pr_test'
eval_batch_list = [f for f in listdir(eval_batch_path) if isfile(join(eval_batch_path, f))]
model_id = 1

validation_on_chunk(model_id, eval_batch_list, 0,classifier1,classifier2)

1 pr_job_45.tsv
21351
21351/21351 [==============================] - 0s 19us/step
8649
2 pr_job_9195.tsv
21333
21333/21333 [==============================] - 0s 21us/step
8667
3 pr_job_9032.tsv
21329
21329/21329 [==============================] - 0s 20us/step
8671
4 pr_job_9272.tsv
21601
21601/21601 [==============================] - 0s 17us/step
8399
5 pr_job_9181.tsv
21214
21214/21214 [==============================] - 0s 17us/step
8786
6 pr_job_9269.tsv
21203
21203/21203 [==============================] - 0s 23us/step
8797
7 pr_job_9227.tsv
21350
21350/21350 [==============================] - 0s 21us/step
8650
8 pr_job_9149.tsv
21068
21068/21068 [==============================] - 0s 18us/step
8932
9 pr_job_9091.tsv
21493
21493/21493 [==============================] - 0s 17us/step
8507
10 pr_job_9064.tsv
21318
21318/21318 [==============================] - 0s 17us/step
8682
11 pr_job_21.tsv
21214
21214/21214 [==============================] - 0s 17us/step
8786
12 pr_job_9008.tsv
2141

21322/21322 [==============================] - 0s 21us/step
8678
93 pr_job_23.tsv
21140
21140/21140 [==============================] - 1s 24us/step
8860
94 pr_job_65.tsv
21652
21652/21652 [==============================] - 0s 22us/step
8348
95 pr_job_9133.tsv
21261
21261/21261 [==============================] - 0s 17us/step
8739
96 pr_job_9073.tsv
21638
21638/21638 [==============================] - 0s 17us/step
8362
97 pr_job_9204.tsv
21511
21511/21511 [==============================] - 0s 19us/step
8489
98 pr_job_9085.tsv
21535
21535/21535 [==============================] - 0s 21us/step
8465
99 pr_job_9075.tsv
21792
21792/21792 [==============================] - 0s 21us/step
8208
100 pr_job_9203.tsv
21578
21578/21578 [==============================] - 0s 20us/step
8422
101 pr_job_61.tsv
21756
21756/21756 [==============================] - 0s 19us/step
8244
102 pr_job_76.tsv
21551
21551/21551 [==============================] - 0s 19us/step
8449
103 pr_job_9239.tsv
21855
21855/21855 [=

8478
183 pr_job_9172.tsv
21146
21146/21146 [==============================] - 0s 17us/step
8854
184 pr_job_9218.tsv
21604
21604/21604 [==============================] - 0s 16us/step
8396
185 pr_job_9009.tsv
21613
21613/21613 [==============================] - 0s 16us/step
8387
186 pr_job_9259.tsv
21479
21479/21479 [==============================] - 0s 17us/step
8521
187 pr_job_9101.tsv
21683
21683/21683 [==============================] - 0s 20us/step
8317
188 pr_job_9200.tsv
21546
21546/21546 [==============================] - 0s 17us/step
8454
189 pr_job_9107.tsv
21587
21587/21587 [==============================] - 0s 16us/step
8413
190 pr_job_34.tsv
21344
21344/21344 [==============================] - 0s 17us/step
8656
191 pr_job_9148.tsv
21351
21351/21351 [==============================] - 0s 17us/step
8649
192 pr_job_9152.tsv
21168
21168/21168 [==============================] - 0s 16us/step
8832
193 pr_job_9212.tsv
21672
21672/21672 [==============================] - 0s 15us/step
8

21644/21644 [==============================] - 0s 21us/step
8356
274 pr_job_9104.tsv
21609
21609/21609 [==============================] - 0s 18us/step
8391
275 pr_job_9110.tsv
21702
21702/21702 [==============================] - 0s 17us/step
8298
276 pr_job_24.tsv
21189
21189/21189 [==============================] - 0s 18us/step
8811
277 pr_job_77.tsv
21643
21643/21643 [==============================] - 0s 19us/step
8357
278 pr_job_9083.tsv
21586
21586/21586 [==============================] - 0s 18us/step
8414
279 pr_job_9162.tsv
21333
21333/21333 [==============================] - 0s 17us/step
8667
280 pr_job_9015.tsv
21570
21570/21570 [==============================] - 0s 19us/step
8430
281 pr_job_9123.tsv
21702
21702/21702 [==============================] - 0s 17us/step
8298
282 pr_job_69.tsv
21613
21613/21613 [==============================] - 0s 17us/step
8387
283 pr_job_9199.tsv
21504
21504/21504 [==============================] - 0s 18us/step
8496
284 pr_job_81.tsv
21603
21603/2